Extraction of head-to-head (H2H) data targeting the 2022 testing datasets, as well as common-opponent (CO) data. The two to be stored in separate files. By construction, for the same matchup, the two different datasets will be disjoint.

### Initialization

In [13]:
# information to specify before running file
target_name = "2022_wta_testing.csv" # file containing matches the data is to be collected for
# list of strings so data from multiple years can be combined, ex. of file name: "atp_matches_2019.csv"
f_names = ["wta_matches_2021.csv","wta_matches_2020.csv","wta_matches_2019.csv"]
h2h_name = "2019-2021_wta_h2h.csv" # name of file to store extracted h2h data in
co_name = "2019-2021_wta_co.csv" # name of file to store extracted co data in
# fields to extract, two of the same to account for the 2 players
# Three extra fields recording number of matches data is based on
# these fields not counted for CO data
f_fields = ["name1","grass_pts_served1","grass_service_pts_won1","grass_pts_received1",
            "grass_receiving_pts_won1","grass_aces1","hard_pts_served1","hard_service_pts_won1",
            "hard_pts_received1","hard_receiving_pts_won1","hard_aces1","clay_pts_served1",
            "clay_service_pts_won1","clay_pts_received1","clay_receiving_pts_won1","clay_aces1",
            "name2","grass_pts_served2","grass_service_pts_won2","grass_pts_received2",
            "grass_receiving_pts_won2","grass_aces2","hard_pts_served2","hard_service_pts_won2",
            "hard_pts_received2","hard_receiving_pts_won2","hard_aces2","clay_pts_served2",
            "clay_service_pts_won2","clay_pts_received2","clay_receiving_pts_won2","clay_aces2",
            "grass_sample_size","hard_sample_size","clay_sample_size"]

In [14]:
import os
import csv

In [15]:
# get paths
path = os.getcwd() # "...\\Code" parallel with "Data Lake"
matches_path = path[:-4]+"Data Lake\\Matches" # folder containing matches to mine
players_path = path[:-4]+"Data Lake\\Players" # folder to store all extracted stats
target_path = path[:-4]+"Data Lake\\Experiments" # folder containing testing targets
t_path = target_path+"\\"+target_name
h2h_path = players_path+"\\"+h2h_name
co_path = players_path+"\\"+co_name

In [16]:
# each match's player stats to be stored in a dict in an overall list
# list will be go by order of matches in the target file
ovr_list = []
# make another one for CO data
# ovr_list_co = ovr_list makes a shallow copy and bleeds h2h into co!!!
ovr_list_co = []

# initializing ovr_list with matches from the target csv file
with open(t_path, 'r') as t:
    
    t_csvreader = csv.reader(t)
    t_fields = next(t_csvreader) # getting fields to identify name columns
    p1_idx = t_fields.index("winner_name")
    p2_idx = t_fields.index("loser_name")
    
    for t_match in t_csvreader:
        p1 = t_match[p1_idx]
        p2 = t_match[p2_idx]
        
        # initialize the dict
        t_dict = {}
        t_dict_co = {}
        for field in f_fields:
            t_dict[field] = 0
            t_dict_co[field] = 0
        t_dict["name1"] = p1
        t_dict["name2"] = p2
        t_dict_co["name1"] = p1
        t_dict_co["name2"] = p2
        
        ovr_list.append(t_dict)
        ovr_list_co.append(t_dict_co)

### H2H Data

In [17]:
# helper functions
def my_int(val):
        """
        some of these entries will be missing, so if an empty string is encountered,
        want it just to be 0
        """
        try:
            return(int(val))
        except ValueError:
            return(0)
        
def update_stats_h2h(list_idx, match, fields, p1w):
    """
    mutates ovr_list - updates stats for the list_idx's entry with data in match
    where p1w indicates whether p1 (name1 in the list entry) was the winner
    """
    
    # indexes of fields needed for statistics
    surface = fields.index("surface")
    w_ace = fields.index("w_ace")
    w_svpt = fields.index("w_svpt")
    w_1stWon = fields.index("w_1stWon")
    w_2ndWon = fields.index("w_2ndWon")
    l_ace = fields.index("l_ace")
    l_svpt = fields.index("l_svpt")
    l_1stWon = fields.index("l_1stWon")
    l_2ndWon = fields.index("l_2ndWon")
    
    if p1w:
        if match[surface] == "Grass":
            ovr_list[list_idx]["grass_pts_served1"] += my_int(match[w_svpt])
            ovr_list[list_idx]["grass_service_pts_won1"] += my_int(match[w_1stWon]) + my_int(match[w_2ndWon])
            ovr_list[list_idx]["grass_pts_received1"] += my_int(match[l_svpt])
            ovr_list[list_idx]["grass_receiving_pts_won1"] += (my_int(match[l_svpt]) - my_int(match[l_1stWon])
                                                               - my_int(match[l_2ndWon]))
            ovr_list[list_idx]["grass_aces1"] += my_int(match[w_ace])
            ovr_list[list_idx]["grass_pts_served2"] += my_int(match[l_svpt])
            ovr_list[list_idx]["grass_service_pts_won2"] += my_int(match[l_1stWon]) + my_int(match[l_2ndWon])
            ovr_list[list_idx]["grass_pts_received2"] += my_int(match[w_svpt])
            ovr_list[list_idx]["grass_receiving_pts_won2"] += (my_int(match[w_svpt]) - my_int(match[w_1stWon])
                                                               - my_int(match[w_2ndWon]))
            ovr_list[list_idx]["grass_aces2"] += my_int(match[l_ace])
            ovr_list[list_idx]["grass_sample_size"] += 1
        elif match[surface] == "Hard":
            ovr_list[list_idx]["hard_pts_served1"] += my_int(match[w_svpt])
            ovr_list[list_idx]["hard_service_pts_won1"] += my_int(match[w_1stWon]) + my_int(match[w_2ndWon])
            ovr_list[list_idx]["hard_pts_received1"] += my_int(match[l_svpt])
            ovr_list[list_idx]["hard_receiving_pts_won1"] += (my_int(match[l_svpt]) - my_int(match[l_1stWon])
                                                               - my_int(match[l_2ndWon]))
            ovr_list[list_idx]["hard_aces1"] += my_int(match[w_ace])
            ovr_list[list_idx]["hard_pts_served2"] += my_int(match[l_svpt])
            ovr_list[list_idx]["hard_service_pts_won2"] += my_int(match[l_1stWon]) + my_int(match[l_2ndWon])
            ovr_list[list_idx]["hard_pts_received2"] += my_int(match[w_svpt])
            ovr_list[list_idx]["hard_receiving_pts_won2"] += (my_int(match[w_svpt]) - my_int(match[w_1stWon])
                                                               - my_int(match[w_2ndWon]))
            ovr_list[list_idx]["hard_aces2"] += my_int(match[l_ace])
            ovr_list[list_idx]["hard_sample_size"] += 1
        elif match[surface] == "Clay":
            ovr_list[list_idx]["clay_pts_served1"] += my_int(match[w_svpt])
            ovr_list[list_idx]["clay_service_pts_won1"] += my_int(match[w_1stWon]) + my_int(match[w_2ndWon])
            ovr_list[list_idx]["clay_pts_received1"] += my_int(match[l_svpt])
            ovr_list[list_idx]["clay_receiving_pts_won1"] += (my_int(match[l_svpt]) - my_int(match[l_1stWon])
                                                               - my_int(match[l_2ndWon]))
            ovr_list[list_idx]["clay_aces1"] += my_int(match[w_ace])
            ovr_list[list_idx]["clay_pts_served2"] += my_int(match[l_svpt])
            ovr_list[list_idx]["clay_service_pts_won2"] += my_int(match[l_1stWon]) + my_int(match[l_2ndWon])
            ovr_list[list_idx]["clay_pts_received2"] += my_int(match[w_svpt])
            ovr_list[list_idx]["clay_receiving_pts_won2"] += (my_int(match[w_svpt]) - my_int(match[w_1stWon])
                                                               - my_int(match[w_2ndWon]))
            ovr_list[list_idx]["clay_aces2"] += my_int(match[l_ace])
            ovr_list[list_idx]["clay_sample_size"] += 1
    else:
        if match[surface] == "Grass":
            ovr_list[list_idx]["grass_pts_served2"] += my_int(match[w_svpt])
            ovr_list[list_idx]["grass_service_pts_won2"] += my_int(match[w_1stWon]) + my_int(match[w_2ndWon])
            ovr_list[list_idx]["grass_pts_received2"] += my_int(match[l_svpt])
            ovr_list[list_idx]["grass_receiving_pts_won2"] += (my_int(match[l_svpt]) - my_int(match[l_1stWon])
                                                               - my_int(match[l_2ndWon]))
            ovr_list[list_idx]["grass_aces2"] += my_int(match[w_ace])
            ovr_list[list_idx]["grass_pts_served1"] += my_int(match[l_svpt])
            ovr_list[list_idx]["grass_service_pts_won1"] += my_int(match[l_1stWon]) + my_int(match[l_2ndWon])
            ovr_list[list_idx]["grass_pts_received1"] += my_int(match[w_svpt])
            ovr_list[list_idx]["grass_receiving_pts_won1"] += (my_int(match[w_svpt]) - my_int(match[w_1stWon])
                                                               - my_int(match[w_2ndWon]))
            ovr_list[list_idx]["grass_aces1"] += my_int(match[l_ace])
            ovr_list[list_idx]["grass_sample_size"] += 1
        elif match[surface] == "Hard":
            ovr_list[list_idx]["hard_pts_served2"] += my_int(match[w_svpt])
            ovr_list[list_idx]["hard_service_pts_won2"] += my_int(match[w_1stWon]) + my_int(match[w_2ndWon])
            ovr_list[list_idx]["hard_pts_received2"] += my_int(match[l_svpt])
            ovr_list[list_idx]["hard_receiving_pts_won2"] += (my_int(match[l_svpt]) - my_int(match[l_1stWon])
                                                               - my_int(match[l_2ndWon]))
            ovr_list[list_idx]["hard_aces2"] += my_int(match[w_ace])
            ovr_list[list_idx]["hard_pts_served1"] += my_int(match[l_svpt])
            ovr_list[list_idx]["hard_service_pts_won1"] += my_int(match[l_1stWon]) + my_int(match[l_2ndWon])
            ovr_list[list_idx]["hard_pts_received1"] += my_int(match[w_svpt])
            ovr_list[list_idx]["hard_receiving_pts_won1"] += (my_int(match[w_svpt]) - my_int(match[w_1stWon])
                                                               - my_int(match[w_2ndWon]))
            ovr_list[list_idx]["hard_aces1"] += my_int(match[l_ace])
            ovr_list[list_idx]["hard_sample_size"] += 1
        elif match[surface] == "Clay":
            ovr_list[list_idx]["clay_pts_served2"] += my_int(match[w_svpt])
            ovr_list[list_idx]["clay_service_pts_won2"] += my_int(match[w_1stWon]) + my_int(match[w_2ndWon])
            ovr_list[list_idx]["clay_pts_received2"] += my_int(match[l_svpt])
            ovr_list[list_idx]["clay_receiving_pts_won2"] += (my_int(match[l_svpt]) - my_int(match[l_1stWon])
                                                               - my_int(match[l_2ndWon]))
            ovr_list[list_idx]["clay_aces2"] += my_int(match[w_ace])
            ovr_list[list_idx]["clay_pts_served1"] += my_int(match[l_svpt])
            ovr_list[list_idx]["clay_service_pts_won1"] += my_int(match[l_1stWon]) + my_int(match[l_2ndWon])
            ovr_list[list_idx]["clay_pts_received1"] += my_int(match[w_svpt])
            ovr_list[list_idx]["clay_receiving_pts_won1"] += (my_int(match[w_svpt]) - my_int(match[w_1stWon])
                                                               - my_int(match[w_2ndWon]))
            ovr_list[list_idx]["clay_aces1"] += my_int(match[l_ace])
            ovr_list[list_idx]["clay_sample_size"] += 1

In [18]:
for file in f_names:
    f_path = matches_path+"\\"+file
    with open(f_path,'r') as f:
        csvreader = csv.reader(f)
        fields = next(csvreader)
        winner_idx = fields.index("winner_name")
        loser_idx = fields.index("loser_name")
        
        # order of the for loops cannot(!!!) be reversed - csvreader is not a normal list
        for match in csvreader:
            for i in range(len(ovr_list)):
                p1 = ovr_list[i]["name1"]
                p2 = ovr_list[i]["name2"]
                if match[winner_idx]==p1 and match[loser_idx]==p2:
                    update_stats_h2h(i,match,fields,p1w=True)
                elif match[loser_idx]==p1 and match[winner_idx]==p2:
                    update_stats_h2h(i,match,fields,p1w=False)

In [19]:
# write ovr_dict into a single csv for R analysis
with open(h2h_path, 'w', newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=f_fields)
    writer.writeheader()
    writer.writerows(ovr_list)

### CO Data

In [20]:
# need to first get a list of common opponents for each matchup
# this will be done by getting 2 lists of played opponents 
# and taking the intersection
# for each matchup, corresponding 2 lists to be stored in separate lists
# initialize the lists of lists
ovr_p1_oppo = []
ovr_p2_oppo = []
for i in range(len(ovr_list_co)):
    p1_oppo = []
    p2_oppo = []
    ovr_p1_oppo.append(p1_oppo)
    ovr_p2_oppo.append(p2_oppo)

# go through matches and add opponents
for file in f_names:
    f_path = matches_path+"\\"+file
    with open(f_path,'r') as f:
        csvreader = csv.reader(f)
        fields = next(csvreader)
        winner_idx = fields.index("winner_name")
        loser_idx = fields.index("loser_name")
        for match in csvreader:
            for i in range(len(ovr_list_co)):
                if match[winner_idx]==ovr_list_co[i]["name1"]:
                    ovr_p1_oppo[i].append(match[loser_idx])
                elif match[loser_idx]==ovr_list_co[i]["name1"]:
                    ovr_p1_oppo[i].append(match[winner_idx])
                if match[winner_idx]==ovr_list_co[i]["name2"]:
                    ovr_p2_oppo[i].append(match[loser_idx])
                elif match[loser_idx]==ovr_list_co[i]["name2"]:
                    ovr_p2_oppo[i].append(match[winner_idx])
                    
# take intersection of each pair of opponent lists for final list of lists
ovr_co_oppo = []
for i in range(len(ovr_list_co)):
    ovr_co_oppo.append(list(set(ovr_p1_oppo[i]).intersection(ovr_p2_oppo[i])))

In [21]:
# we can see the lists of common opponents produced, for example:
k = 2
print("Common opponents for "+ovr_list_co[k]["name1"]+" and "+ovr_list_co[k]["name2"]+":")
ovr_co_oppo[k]
# ovr_list_co[k]

Common opponents for Barbora Krejcikova and Victoria Azarenka:


['Daria Kasatkina',
 'Maria Sakkari',
 'Sorana Cirstea',
 'Yulia Putintseva',
 'Karolina Pliskova',
 'Anett Kontaveit',
 'Andrea Petkovic',
 'Zarina Diyas',
 'Paula Badosa',
 'Iga Swiatek',
 'Angelique Kerber',
 'Ashleigh Barty',
 'Dayana Yastremska',
 'Sloane Stephens',
 'Harmony Tan',
 'Aryna Sabalenka',
 'Ekaterina Alexandrova',
 'Belinda Bencic',
 'Clara Tauson',
 'Sofia Kenin',
 'Simona Halep',
 'Jelena Ostapenko',
 'Svetlana Kuznetsova',
 'Anastasia Pavlyuchenkova',
 'Elina Svitolina',
 'Aliaksandra Sasnovich',
 'Caroline Garcia',
 'Tereza Martincova',
 'Garbine Muguruza']

In [22]:
# now we can do something similar to the H2H data extraction above
# but this time the data of the common opponents isn't extracted
# so the update_stats function will be modified
def update_stats_co(list_idx, match, fields, p1, pw):
    """
    mutates ovr_list_co - updates stats for the list_idx'th entry with data in match
    where boolean p1 indicates whether p1 (name1 in the list entry) or p2 is to be updated
    and boolean pw indicates whether the player being updated won or lost the match
    """
    
    # indexes of fields needed for statistics
    surface = fields.index("surface")
    w_ace = fields.index("w_ace")
    w_svpt = fields.index("w_svpt")
    w_1stWon = fields.index("w_1stWon")
    w_2ndWon = fields.index("w_2ndWon")
    l_ace = fields.index("l_ace")
    l_svpt = fields.index("l_svpt")
    l_1stWon = fields.index("l_1stWon")
    l_2ndWon = fields.index("l_2ndWon")
    
    if p1:
        if pw:
            if match[surface] == "Grass":
                ovr_list_co[list_idx]["grass_pts_served1"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["grass_service_pts_won1"] += (my_int(match[w_1stWon]) 
                                                                    + my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["grass_pts_received1"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["grass_receiving_pts_won1"] += (my_int(match[l_svpt]) 
                                                                      - my_int(match[l_1stWon])
                                                                      - my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["grass_aces1"] += my_int(match[w_ace])
            elif match[surface] == "Hard":
                ovr_list_co[list_idx]["hard_pts_served1"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["hard_service_pts_won1"] += (my_int(match[w_1stWon]) 
                                                                   + my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["hard_pts_received1"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["hard_receiving_pts_won1"] += (my_int(match[l_svpt]) 
                                                                     - my_int(match[l_1stWon])
                                                                     - my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["hard_aces1"] += my_int(match[w_ace])
            elif match[surface] == "Clay":
                ovr_list_co[list_idx]["clay_pts_served1"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["clay_service_pts_won1"] += (my_int(match[w_1stWon]) 
                                                                   + my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["clay_pts_received1"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["clay_receiving_pts_won1"] += (my_int(match[l_svpt]) 
                                                                     - my_int(match[l_1stWon])
                                                                     - my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["clay_aces1"] += my_int(match[w_ace])
        else:
            if match[surface] == "Grass":
                ovr_list_co[list_idx]["grass_pts_served1"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["grass_service_pts_won1"] += (my_int(match[l_1stWon]) 
                                                                    + my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["grass_pts_received1"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["grass_receiving_pts_won1"] += (my_int(match[w_svpt]) 
                                                                      - my_int(match[w_1stWon])
                                                                      - my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["grass_aces1"] += my_int(match[l_ace])
            if match[surface] == "Hard":
                ovr_list_co[list_idx]["hard_pts_served1"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["hard_service_pts_won1"] += (my_int(match[l_1stWon]) 
                                                                   + my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["hard_pts_received1"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["hard_receiving_pts_won1"] += (my_int(match[w_svpt]) 
                                                                     - my_int(match[w_1stWon])
                                                                     - my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["hard_aces1"] += my_int(match[l_ace])
            if match[surface] == "Clay":
                ovr_list_co[list_idx]["clay_pts_served1"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["clay_service_pts_won1"] += (my_int(match[l_1stWon]) 
                                                                   + my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["clay_pts_received1"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["clay_receiving_pts_won1"] += (my_int(match[w_svpt]) 
                                                                     - my_int(match[w_1stWon])
                                                                     - my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["clay_aces1"] += my_int(match[l_ace])
    else:
        if pw:
            if match[surface] == "Grass":
                ovr_list_co[list_idx]["grass_pts_served2"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["grass_service_pts_won2"] += (my_int(match[w_1stWon]) 
                                                                    + my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["grass_pts_received2"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["grass_receiving_pts_won2"] += (my_int(match[l_svpt]) 
                                                                      - my_int(match[l_1stWon])
                                                                      - my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["grass_aces2"] += my_int(match[w_ace])
            elif match[surface] == "Hard":
                ovr_list_co[list_idx]["hard_pts_served2"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["hard_service_pts_won2"] += (my_int(match[w_1stWon]) 
                                                                   + my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["hard_pts_received2"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["hard_receiving_pts_won2"] += (my_int(match[l_svpt]) 
                                                                     - my_int(match[l_1stWon])
                                                                     - my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["hard_aces2"] += my_int(match[w_ace])
            elif match[surface] == "Clay":
                ovr_list_co[list_idx]["clay_pts_served2"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["clay_service_pts_won2"] += (my_int(match[w_1stWon]) 
                                                                   + my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["clay_pts_received2"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["clay_receiving_pts_won2"] += (my_int(match[l_svpt]) 
                                                                     - my_int(match[l_1stWon])
                                                                     - my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["clay_aces2"] += my_int(match[w_ace])
        else:
            if match[surface] == "Grass":
                ovr_list_co[list_idx]["grass_pts_served2"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["grass_service_pts_won2"] += (my_int(match[l_1stWon]) 
                                                                    + my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["grass_pts_received2"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["grass_receiving_pts_won2"] += (my_int(match[w_svpt]) 
                                                                      - my_int(match[w_1stWon])
                                                                      - my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["grass_aces2"] += my_int(match[l_ace])
            if match[surface] == "Hard":
                ovr_list_co[list_idx]["hard_pts_served2"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["hard_service_pts_won2"] += (my_int(match[l_1stWon]) 
                                                                   + my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["hard_pts_received2"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["hard_receiving_pts_won2"] += (my_int(match[w_svpt]) 
                                                                     - my_int(match[w_1stWon])
                                                                     - my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["hard_aces2"] += my_int(match[l_ace])
            if match[surface] == "Clay":
                ovr_list_co[list_idx]["clay_pts_served2"] += my_int(match[l_svpt])
                ovr_list_co[list_idx]["clay_service_pts_won2"] += (my_int(match[l_1stWon]) 
                                                                   + my_int(match[l_2ndWon]))
                ovr_list_co[list_idx]["clay_pts_received2"] += my_int(match[w_svpt])
                ovr_list_co[list_idx]["clay_receiving_pts_won2"] += (my_int(match[w_svpt]) 
                                                                     - my_int(match[w_1stWon])
                                                                     - my_int(match[w_2ndWon]))
                ovr_list_co[list_idx]["clay_aces2"] += my_int(match[l_ace])

In [23]:
for file in f_names:
    f_path = matches_path+"\\"+file
    with open(f_path,'r') as f:
        csvreader = csv.reader(f)
        fields = next(csvreader)
        winner_idx = fields.index("winner_name")
        loser_idx = fields.index("loser_name")
        
        for match in csvreader:
            for i in range(len(ovr_list_co)):
                p1 = ovr_list_co[i]["name1"]
                p2 = ovr_list_co[i]["name2"]
                if match[winner_idx]==p1 and match[loser_idx] in ovr_co_oppo[i]:
                    update_stats_co(i,match,fields,p1=True,pw=True)
                elif match[loser_idx]==p1 and match[winner_idx] in ovr_co_oppo[i]:
                    update_stats_co(i,match,fields,p1=True,pw=False)
                elif match[winner_idx]==p2 and match[loser_idx] in ovr_co_oppo[i]:
                    update_stats_co(i,match,fields,p1=False,pw=True)
                elif match[loser_idx]==p2 and match[winner_idx] in ovr_co_oppo[i]:
                    update_stats_co(i,match,fields,p1=False,pw=False)

In [24]:
# write ovr_dict into a single csv for R analysis
with open(co_path, 'w', newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=f_fields)
    writer.writeheader()
    writer.writerows(ovr_list_co)